# Intermediate LePHARE use case

In the minimal photoz run example notebook we demonstrated a run on the COSMOS example in order to show the most basic LePHARE functionality.

In this notebook we want to walk through a typical use case where the user wishes to run on a new catalogue with a new set of filters.

We will be looking at the same COSMOS data set but only use the ugrizy bands. We will also use the Spanish Virtual Observatory to get the filters rather than relying on the versions in the LePHARE auxiliary data repository.

In [ ]:
import lephare as lp
from astropy.table import Table
import astropy.units as u
import numpy as np
import os
from collections import OrderedDict
from matplotlib import pylab as plt
import time

%matplotlib inline

## Update the config
We will start with the COSMOS configuration as a basis. We will update the various keywords. We use the default which is shipped with lephare. You could also download the eaxmple text file config from [here](https://github.com/lephare-photoz/lephare-data/blob/main/examples/COSMOS.para).

In [ ]:
config = lp.default_cosmos_config.copy()

# You could also load this from a local text file:
# !curl -s -o COSMOS.para https://github.com/lephare-photoz/lephare-data/blob/main/examples/COSMOS.para
# config = lp.read_config("./COSMOS.para")

config.update(
    {
        # The following measurements will correspond to all filters. We could have an array of values for each.
        "ERR_SCALE": "0.02",
        "ERR_FACTOR": "1.5",
        "FILTER_CALIB": "0",
        "FILTER_FILE": "filter_test",
        # For a quick demonstration we use a very sparse redshift grid. DO NOT USE FOR SCIENCE!
        # Comment out the following line to improve results.
        "Z_STEP": "0.5,0.,7.",
    }
)

## Download the required SEDs and additional extinction laws
We will need the same set of SEDs and other files required for the COSMOS example so will download those using the automated download functionality.

In [ ]:
lp.data_retrieval.get_auxiliary_data(
    keymap=config,
    # The additional extinction laws for galaxies are not in the principle config
    # so we must add them to be downloaded:
    additional_files=[
        "ext/SMC_prevot.dat",
        "ext/SB_calzetti.dat",
        "ext/SB_calzetti_bump1.dat",
        "ext/SB_calzetti_bump2.dat",
        # We also want the example cosmos catalogue to experiment with
        "examples/COSMOS.in",
    ],
)

## Getting new filters

First lets get the filters we need from the SVO

In [ ]:
# We can get filters from the Spanish Virtual Observatory using the correct ids available.
# See ids here http://svo2.cab.inta-csic.es/theory/fps/
svo_filters = [
    "CFHT/MegaCam.u",
    "Subaru/HSC.g",
    "Subaru/HSC.r",
    "Subaru/HSC.i",
    "Subaru/HSC.z",
    "Subaru/HSC.y",
]
# We make a list of lephare.flt objects
filterLib = [lp.FilterSvc.from_svo(n, i) for n, i in enumerate(svo_filters)]

In [ ]:
# We can write the filter file to the LEPHAREWORK director using the lephare format:
filter_output = os.path.join(os.environ["LEPHAREWORK"], "filt", config["FILTER_FILE"])
lp.write_output_filter(f"{filter_output}_svo.dat", f"{filter_output}_svo.doc", filterLib)

In [ ]:
# Check the files are there
!ls -al $LEPHAREWORK/filt/filter_test_svo*

We can also write the filters locally and update the config to take them from there

In [ ]:
!mkdir filt

In [ ]:
for n, f in enumerate(svo_filters):
    data = Table()
    data["wavelength"] = filterLib[n].data()[0]
    data["transmission"] = filterLib[n].data()[1]
    plt.plot(data["wavelength"], data["transmission"], label=f)
    directory = f"./filt/{f.split('/')[0]}"
    if not os.path.exists(directory):
        os.makedirs(directory)
    data.write(f"./filt/{f}", format="ascii", overwrite=True)
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5))
plt.xlabel("Wavelength [Angstrom]")
plt.ylabel("Transmission")

In [ ]:
!ls -al filt/*

## Update filter list

to use these new filters we need to update the config and set the filter directory to their location

In [ ]:
config.update(
    {
        # We now set the FILTER_REP keyword to that location to ensure that lephare looks in the right place
        "FILTER_REP": os.path.join(os.getcwd(), "filt"),
        # We also update the names to the new SVO names
        "FILTER_LIST": ",".join(svo_filters),
        "SPEC_OUT": "YES",  # We want to look at the model spectra
    }
)

In [ ]:
config

In [ ]:
config["FILTER_REP"], config["FILTER_LIST"]

### Set object specific config values 

In order to get better results we often also want to use different config values for stars, galaxies and qso.

We therefore make override dictionaries for each type.

In [ ]:
# We leave stars as before
star_overrides = {}

# For galaxies we want to use a different set of extinction laws and other keyword values
gal_overrides = {
    "MOD_EXTINC": "18,26,26,33,26,33,26,33",
    "EXTINC_LAW": "SMC_prevot.dat,SB_calzetti.dat,SB_calzetti_bump1.dat,SB_calzetti_bump2.dat",
    "EM_LINES": "EMP_UV",
    "EM_DISPERSION": "0.5,0.75,1.,1.5,2.",
}

qso_overrides = {
    "MOD_EXTINC": "0,1000",
    "EB_V": "0.,0.1,0.2,0.3",
    "EXTINC_LAW": "SB_calzetti.dat",
}

## Run prepare

The prepare method runs filter, sedtolib, and mag_gal. These are the key preparatory stages that calculate the filters in the LePHARE format, calculate the library of SEDs and finaly calculate the library of magnitudes for all the models

In [ ]:
lp.prepare(
    config,
    star_config=star_overrides,
    gal_config=gal_overrides,
    qso_config=qso_overrides,
)

## Run process

We need to take the input catalogue and organise it in the correct format for LePHARE. By default this is done using column order.

Because we are removing most of the fluxes we now just need the ugrizy bands

In [ ]:
# Load the full cosmos example we downloaded at the start
cosmos_full = Table.read(f"{lp.LEPHAREDIR}/examples/COSMOS.in", format="ascii")
# Lets just look at the first 100 specz to be fast for now
specz_colname = cosmos_full.colnames[-2]
cosmos_full = cosmos_full[cosmos_full[specz_colname] > 0][:100]

In [ ]:
# Look at the original filter order to see which columns in the original catalogue correspond to the new filters
lp.default_cosmos_config.copy()["FILTER_LIST"]

In [ ]:
# You will need to set the table columns in order:
# id, flux0, err0, flux1, err1,..., context, zspec, arbitrary_string
input_table = Table()
# The id is in the first column
input_table["id"] = cosmos_full[cosmos_full.colnames[0]]
for n, b in enumerate(svo_filters):
    # The ugrizy fluxes and errors are in cols 3 to 14
    f_col = cosmos_full.colnames[2 * n + 3]
    ferr_col = cosmos_full.colnames[2 * n + 4]
    input_table[f"f_{b}"] = cosmos_full[f_col]
    input_table[f"f_{b}"].unit = u.erg / u.s / u.cm**2 / u.Hz
    input_table[f"ferr_{b}"] = cosmos_full[ferr_col]
    input_table[f"ferr_{b}"].unit = u.erg / u.s / u.cm**2 / u.Hz
# The context is a binary flag. Here we set it to use all filters.
input_table["context"] = np.sum(2 ** np.arange(len(svo_filters)))
input_table["zspec"] = cosmos_full[specz_colname]
input_table["string_data"] = "arbitrary_info"

Finally we repeat the main run stages prepare and process to get the results

In [ ]:
# Calculate the photometric redshifts
output, pdfs, zgrid = lp.process(config, input_table)

In [ ]:
# the output is an astropy tabel that can be manipulated in the standard ways.
output[:5]

In [ ]:
plt.hist(output["Z_BEST"], bins=20)
plt.xlabel("redshift")

## Lower level functionality

The example above uses the high level *prepare* and *process* methods. It is also possible to have much more control over every stage of the run by using lower level classes. Here we will briefly introduce this functionality by running photoz on a single object and showing it's best fit model.

In [ ]:
# First we create a PhotoZ object from the config
config["SPEC_OUT"] = "save_spec"  # Set location for output spectra
photz = lp.PhotoZ(lp.all_types_to_keymap(config))

In [ ]:
# Then we make a list of the sources as lephare onesource objects
photozlist = []
fluxes = input_table[[f"f_{b}" for b in svo_filters]]
efluxes = input_table[[f"ferr_{b}" for b in svo_filters]]
# Lets just look at the first five objects for now
n = 5
for i in range(n):
    oneObj = lp.onesource(i, photz.gridz)
    oneObj.readsource(
        str(i),
        list(fluxes[i]),
        list(efluxes[i]),
        int(np.sum(2 ** np.arange(len(svo_filters)))),
        input_table["zspec"][i],
        " ",
    )
    photz.prep_data(oneObj)
    photozlist.append(oneObj)
print("Number of sources to be analysed: ", len(photozlist))

In [ ]:
os.getenv("LEPHAREDIR")

In [ ]:
!mkdir save_spec

In [ ]:
# then we run the photoz on those sources
zeros = [0.0 for f in config["FILTER_LIST"].split(",")]  # Use zero offset
photz.run_photoz(photozlist[:n], zeros, zeros)

In [ ]:
# photz.build_output_tables(photozlist[:n], para_out=None, filename="outputpython.fits")

In [ ]:
photz.write_outputs(photozlist[:n], int(time.time()))

In [ ]:
listname = [f for f in os.listdir("save_spec") if os.path.isfile(os.path.join("save_spec", f))]
for namefile in listname:
    lp.plotspec("save_spec/" + str(namefile))